In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this task you will build a classifier for Rock-Paper-Scissors 
# based on the rps dataset.
#
# IMPORTANT: Your final layer should be as shown, do not change the
# provided code, or the tests may fail
#
# IMPORTANT: Images will be tested as 150x150 with 3 bytes of color depth
# So ensure that your input layer is designed accordingly, or the tests
# may fail. 
#
# NOTE THAT THIS IS UNLABELLED DATA. 
# You can use the ImageDataGenerator to automatically label it
# and we have provided some starter code.

# =================================================== #
# =================================================== #
# ====== 모델링 할 때는 포함, 시험 볼때는 제거 ====== #
# 텐서플로우 버전 충돌로 인하여 
# 반드시 tensorflow 버전을 2.1.0 버전 설치 후 
# 모델작업 진행 해주셔야 합니다.
# 시험보실 때는 아래 
# !pip install tensorflow==2.1.0 명령어 필요 없습니다
# 텐서플로우 2.1.0 버전 재설치 코드
!pip install tensorflow==2.1.0

# ====== 모델링 할 때는 포함, 시험 볼때는 제거 ====== #
# =================================================== #

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 3 - rps
# val_loss: 0.0871
# val_acc: 0.97
# =================================================== #
# =================================================== #

import urllib.request
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    urllib.request.urlretrieve(url, 'rps.zip')
    local_zip = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/')
    zip_ref.close()


    TRAINING_DIR = "tmp/rps/"
    # YOUR CODE HERE)
    training_datagen = ImageDataGenerator(
      rescale=1. / 255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest', 
      validation_split=0.2
      )

    #train_generator = # YOUR CODE HERE
    training_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                              batch_size=32, 
                                                              target_size=(150, 150), 
                                                              class_mode='categorical', 
                                                              subset='training',
                                                            )

    validation_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                              batch_size=32, 
                                                              target_size=(150, 150), 
                                                              class_mode='categorical',
                                                              subset='validation', 
                                                            )


    model = tf.keras.models.Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2), 
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Conv2D(128, (3, 3), activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Conv2D(256, (3, 3), activation='relu'),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Flatten(), 
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),

    # YOUR CODE HERE, BUT END WITH A 3 Neuron Dense, activated by softmax
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.summary()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    checkpoint_path = "tmp_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                                save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss', 
                                verbose=1)

    model.fit(training_generator, 
                        validation_data=(validation_generator),
                        epochs=20,
                        callbacks=[checkpoint],
                        )

    model.load_weights(checkpoint_path)

    return model



# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-rps.h5")


Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 72, 72, 128)       73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 70, 70, 128)       147584    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 33, 33, 128)       147584    
_____________________________________

KeyboardInterrupt: ignored